In [1]:
import h5py
import numpy as np
import bcolz
import allel
allel.__version__

'1.0.0.dev12'

In [2]:
# data from http://www.malariagen.net/data/ag1000g-phase1-ar3
callset = h5py.File('/data/coluzzi/ag1000g/data/phase1/release/AR3/variation/main/hdf5/ag1000g.phase1.ar3.pass.h5',
                    mode='r')

## GenotypeChunkedArray

In [3]:
genotype = allel.GenotypeChunkedArray(callset['3L/calldata/genotype'])
genotype

,0,1,2,3,4,...,760,761,762,763,764
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0


In [4]:
%time genotype.max()

CPU times: user 46.1 s, sys: 240 ms, total: 46.3 s
Wall time: 46.3 s


3

In [4]:
genotype_bcolzmem = genotype.copy(stop=1000000, storage='bcolzmem')
%time genotype_bcolzmem.max()

CPU times: user 2.98 s, sys: 48 ms, total: 3.03 s
Wall time: 2.34 s


3

In [5]:
genotype_bcolztmp = genotype.copy(stop=1000000, storage='bcolztmp', dir='/magnetic')
%time genotype_bcolztmp.max()

CPU times: user 2.99 s, sys: 92 ms, total: 3.08 s
Wall time: 2.44 s


3

In [6]:
genotype_bcolzmem_zlib1 = genotype.copy(stop=1000000, storage='bcolzmem_zlib1')
%time genotype_bcolzmem_zlib1.max()

CPU times: user 5.49 s, sys: 56 ms, total: 5.55 s
Wall time: 3 s


3

In [7]:
genotype_bcolztmp_zlib1 = genotype.copy(stop=1000000, storage='bcolztmp_zlib1', dir='/magnetic')
%time genotype_bcolztmp_zlib1.max()

CPU times: user 5.46 s, sys: 48 ms, total: 5.5 s
Wall time: 3.11 s


3

In [8]:
genotype_hdf5mem_zlib1 = genotype.copy(stop=1000000, storage='hdf5mem_zlib1')
%time genotype_hdf5mem_zlib1.max()

CPU times: user 4.35 s, sys: 0 ns, total: 4.35 s
Wall time: 4.34 s


3

In [9]:
genotype_hdf5tmp_zlib1 = genotype.copy(stop=1000000, storage='hdf5tmp_zlib1', dir='/magnetic')
%time genotype_hdf5tmp_zlib1.max()

CPU times: user 4.32 s, sys: 4 ms, total: 4.33 s
Wall time: 4.32 s


3

### Tune default HDF5 chunk size

In [14]:
genotype.copy(storage='bcolzmem').chunklen

2741

In [10]:
genotype_bcolzmem.chunklen

1370

In [11]:
genotype_hdf5mem_zlib1.chunks

(685, 765, 2)

In [11]:
for chunksize in 2**15, 2**16, 2**17, 2**18, 2**19, 2**20, 2**21, 2**22, 2**23:
    chunklen = chunksize // (genotype.shape[1] * genotype.shape[2])
    chunks = (chunklen,) + genotype.shape[1:]
    print(chunksize, chunklen)
    genotype_hdf5mem_zlib1_bigchunks = genotype.copy(stop=1000000, storage='hdf5mem_zlib1', chunks=chunks)
    %time genotype_hdf5mem_zlib1_bigchunks.max()

32768 21
CPU times: user 11.7 s, sys: 0 ns, total: 11.7 s
Wall time: 11.7 s
65536 42
CPU times: user 8.16 s, sys: 0 ns, total: 8.16 s
Wall time: 8.15 s
131072 85
CPU times: user 5.77 s, sys: 0 ns, total: 5.77 s
Wall time: 5.77 s
262144 171
CPU times: user 4.83 s, sys: 4 ms, total: 4.84 s
Wall time: 4.83 s
524288 342
CPU times: user 4.44 s, sys: 0 ns, total: 4.44 s
Wall time: 4.44 s
1048576 685
CPU times: user 4.28 s, sys: 0 ns, total: 4.28 s
Wall time: 4.27 s
2097152 1370
CPU times: user 4.14 s, sys: 0 ns, total: 4.14 s
Wall time: 4.14 s
4194304 2741
CPU times: user 4.1 s, sys: 0 ns, total: 4.1 s
Wall time: 4.09 s
8388608 5482
CPU times: user 4 s, sys: 0 ns, total: 4 s
Wall time: 3.99 s


### Excercise methods

In [12]:
%time genotype_bcolzmem.is_called()

CPU times: user 7.03 s, sys: 56 ms, total: 7.08 s
Wall time: 2.82 s


ChunkedArray((1000000, 765), bool, bcolz.carray_ext.carray)

In [13]:
%time genotype_hdf5mem_zlib1.is_called(storage='hdf5mem_zlib1')

CPU times: user 11 s, sys: 92 ms, total: 11.1 s
Wall time: 7.66 s


ChunkedArray((1000000, 765), bool, h5py._hl.dataset.Dataset)

In [14]:
%time genotype_bcolzmem.count_het()

CPU times: user 9.34 s, sys: 48 ms, total: 9.39 s
Wall time: 3.63 s


28029171

In [15]:
%time genotype_hdf5mem_zlib1.count_het()

CPU times: user 11 s, sys: 120 ms, total: 11.1 s
Wall time: 6.24 s


28029171

In [16]:
%time genotype_bcolzmem.count_alleles()

CPU times: user 7.83 s, sys: 48 ms, total: 7.88 s
Wall time: 6.37 s


,0,1,2,3
0,1527,3,0,0
1,1529,1,0,0
2,1528,2,0,0
3,1528,2,0,0
4,1526,4,0,0


In [12]:
%time genotype_bcolzmem_zlib1.count_alleles()

CPU times: user 12.4 s, sys: 64 ms, total: 12.4 s
Wall time: 7.56 s


,0,1,2,3
0,1527,3,0,0
1,1529,1,0,0
2,1528,2,0,0
3,1528,2,0,0
4,1526,4,0,0


In [17]:
%time genotype_hdf5mem_zlib1.count_alleles(storage='hdf5mem_zlib1')

CPU times: user 10.8 s, sys: 4 ms, total: 10.8 s
Wall time: 10.8 s


,0,1,2,3
0,1527,3,0,0
1,1529,1,0,0
2,1528,2,0,0
3,1528,2,0,0
4,1526,4,0,0


In [10]:
acs = genotype_bcolzmem.count_alleles_subpops(subpops={'pop1': list(range(100)),
                                                       'pop2': list(range(100, 200))},
                                              max_allele=3)
acs

pop1,pop2
[200 0 0 0],[200 0 0 0]
[199 1 0 0],[200 0 0 0]
[200 0 0 0],[200 0 0 0]
[198 2 0 0],[200 0 0 0]
[196 4 0 0],[200 0 0 0]


In [11]:
acs['pop1']

,0,1,2,3
0,200,0,0,0
1,199,1,0,0
2,200,0,0,0
3,198,2,0,0
4,196,4,0,0


## Sysadmin

In [20]:
!yes | pip3 uninstall scikit-allel

Uninstalling scikit-allel-1.0.0.dev11:
  /usr/local/lib/python3.4/dist-packages/scikit_allel-1.0.0.dev11-py3.4-linux-x86_64.egg
Proceed (y/n)?   Successfully uninstalled scikit-allel-1.0.0.dev11
yes: standard output: Broken pipe
yes: write error


In [21]:
!cd /src/github/cggh/scikit-allel/ && python3 setup.py install

running install
Checking .pth file support in /usr/local/lib/python3.4/dist-packages/
/usr/bin/python3 -E -c pass
TEST PASSED: /usr/local/lib/python3.4/dist-packages/ appears to support .pth files
running bdist_egg
running egg_info
writing scikit_allel.egg-info/PKG-INFO
writing top-level names to scikit_allel.egg-info/top_level.txt
writing dependency_links to scikit_allel.egg-info/dependency_links.txt
reading manifest file 'scikit_allel.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'scikit_allel.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying ./allel/__init__.py -> build/lib.linux-x86_64-3.4/allel
copying ./allel/model/chunked.py -> build/lib.linux-x86_64-3.4/allel/model
running build_ext
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/allel
copying build/lib.linux-x86_64-3.4/allel/compat.py -> build/bdist.linux-x86_64/egg/allel
creating build/bdi